In [1]:
import pandas as pd
import json
import os
import re

In [2]:
df0605 = pd.read_parquet('data/api_fetches/20240605.parquet')
df0606 = pd.read_parquet('data/api_fetches/20240606.parquet')

In [3]:
df = pd.concat([df0605, df0606])

In [4]:
df.describe()

,start_time,arrival,departure,latitude,longitude,bearing,timestamp,precipitationProbability,iceAccumulation,rainAccumulation,sleetAccumulation,snowAccumulation,snowDepth,temperature
count,6709888,6.000476e+06,5.899493e+06,6.719951e+06,6.719951e+06,6.719951e+06,6719951,6719951.0,6719951.0,6719951.0,6719951.0,6719951.0,6719951.0,6.719951e+06
mean,2024-06-07 04:16:58.381601792,1.717663e+09,1.717663e+09,4.235376e+01,-7.108093e+01,1.278876e+02,2024-06-06 08:30:46.980247552,0.0,0.0,0.0,0.0,0.0,0.0,2.421713e+01
min,2024-06-06 13:25:00,1.717614e+09,1.717614e+09,4.158184e+01,-7.184799e+01,0.000000e+00,2024-06-05 18:22:04,0.0,0.0,0.0,0.0,0.0,0.0,1.631000e+01
25%,2024-06-06 17:01:00,1.717624e+09,1.717624e+09,4.231030e+01,-7.111921e+01,0.000000e+00,2024-06-05 21:19:33,0.0,0.0,0.0,0.0,0.0,0.0,2.063000e+01
50%,2024-06-06 22:22:00,1.717637e+09,1.717637e+09,4.235271e+01,-7.108096e+01,1.050000e+02,2024-06-06 02:30:55,0.0,0.0,0.0,0.0,0.0,0.0,2.163000e+01
75%,2024-06-07 16:40:00,1.717709e+09,1.717709e+09,4.239778e+01,-7.104568e+01,2.250000e+02,2024-06-06 20:54:55,0.0,0.0,0.0,0.0,0.0,0.0,2.850000e+01
max,2024-06-07 23:54:00,1.717736e+09,1.717736e+09,4.279715e+01,-7.062737e+01,3.600000e+02,2024-06-07 03:42:00,0.0,0.0,0.0,0.0,0.0,0.0,2.931000e+01
std,NaN,4.236659e+04,4.235445e+04,7.599935e-02,7.727363e-02,1.168937e+02,NaN,0.0,0.0,0.0,0.0,0.0,0.0,4.222066e+00


In [5]:
df.head()

,trip_id,start_time,start_date,stop_id,arrival,departure,vehicle_id,latitude,longitude,bearing,timestamp,collection_time,precipitationProbability,iceAccumulation,rainAccumulation,sleetAccumulation,snowAccumulation,snowDepth,temperature
0,62145526_2,2024-06-06 15:01:00,20240605,10642,NaN,1.717614e+09,y1655,42.288845,-71.125549,0,2024-06-05 18:59:51,20240605_150000,0,0,0,0,0,0,29.31
1,62145526_2,2024-06-06 15:01:00,20240605,10642,NaN,1.717614e+09,y1655,42.288845,-71.125549,0,2024-06-05 18:59:51,20240605_150000,0,0,0,0,0,0,29.31
2,62145526_2,2024-06-06 15:01:00,20240605,10642,NaN,1.717614e+09,y1655,42.288845,-71.125549,0,2024-06-05 18:59:51,20240605_150000,0,0,0,0,0,0,29.31
3,62145526_2,2024-06-06 15:01:00,20240605,10642,NaN,1.717614e+09,y1655,42.288845,-71.125549,0,2024-06-05 18:59:51,20240605_150000,0,0,0,0,0,0,29.31
4,62145526_2,2024-06-06 15:01:00,20240605,10642,NaN,1.717614e+09,y1655,42.288845,-71.125549,0,2024-06-05 18:59:51,20240605_150000,0,0,0,0,0,0,29.31


In [6]:
stops = pd.read_csv('data/stops.csv')

In [8]:
stops.head()

,Unnamed: 0,address,at_street,description,latitude,location_type,longitude,municipality,name,on_street,platform_code,platform_name,vehicle_type,wheelchair_boarding
0,3782,NaN,NaN,NaN,42.201868,0,-71.007149,Braintree,Franklin St @ Washington St,Franklin Street,NaN,NaN,3.0,1
1,49810,NaN,NaN,NaN,42.503598,0,-71.239766,Bedford,MITRE Corp Inner Rd,NaN,NaN,NaN,3.0,1
2,8328,NaN,Milton Street,NaN,42.240706,0,-71.143445,Boston,1 W Milton St opp Ernest Ave,West Milton Street,NaN,NaN,3.0,1
3,14586,NaN,NaN,NaN,42.526045,0,-70.951632,Peabody,10 Centennial Drive - Boston Children's @ Peabody,NaN,NaN,NaN,3.0,1
4,24586,NaN,NaN,NaN,42.526993,0,-70.954649,Peabody,10 Centennial Drive - West,NaN,NaN,NaN,3.0,1


In [9]:
df['latitude'] = df['latitude'].round(5)
df['longitude'] = df['longitude'].round(5)
stops['latitude'] = stops['latitude'].round(5)
stops['longitude'] = stops['longitude'].round(5)

In [10]:
stops.describe()

,latitude,location_type,longitude,vehicle_type,wheelchair_boarding
count,9124.000000,9975.000000,9124.000000,7754.000000,9975.000000
mean,42.354826,0.582256,-71.079892,2.864328,0.988271
std,0.100440,1.136214,0.103186,0.526025,0.297309
min,41.581100,0.000000,-71.848490,0.000000,0.000000
25%,42.291103,0.000000,-71.133843,3.000000,1.000000
50%,42.358625,0.000000,-71.075895,3.000000,1.000000
75%,42.416170,0.000000,-71.023057,3.000000,1.000000
max,42.797840,3.000000,-70.276580,4.000000,2.000000


In [11]:
stops['Unnamed: 0'].nunique()

9975

In [12]:
len(stops)

9975

In [14]:
stops = stops.rename(columns={'Unnamed: 0': 'id'})

In [15]:
stops_trimmed = stops[['id', 'latitude', 'longitude', 'location_type', 'vehicle_type']]

In [16]:
stops_trimmed

,id,latitude,longitude,location_type,vehicle_type
0,3782,42.20187,-71.00715,0,3.0
1,49810,42.50360,-71.23977,0,3.0
2,8328,42.24071,-71.14344,0,3.0
3,14586,42.52604,-70.95163,0,3.0
4,24586,42.52699,-70.95465,0,3.0
...,...,...,...,...,...
9970,WR-0062-02,42.45173,-71.06938,0,2.0
9971,WR-0062-B2,42.45173,-71.06724,0,3.0
9972,WR-0062-B3,42.45156,-71.06747,0,3.0
9973,WR-0062-B0,42.45191,-71.07011,0,3.0


In [18]:
df = pd.merge(df, stops_trimmed, how='outer', on=['latitude', 'longitude'])

In [19]:
df['stop_id']

0          10642
1          10642
2          10642
3          10642
4          10642
           ...  
6730070      NaN
6730071      NaN
6730072      NaN
6730073      NaN
6730074      NaN
Name: stop_id, Length: 6730075, dtype: object

In [20]:
df.head()

,trip_id,start_time,start_date,stop_id,arrival,departure,vehicle_id,latitude,longitude,bearing,...,precipitationProbability,iceAccumulation,rainAccumulation,sleetAccumulation,snowAccumulation,snowDepth,temperature,id,location_type,vehicle_type
0,62145526_2,2024-06-06 15:01:00,20240605,10642,NaN,1.717614e+09,y1655,42.28885,-71.12555,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,29.31,NaN,NaN,NaN
1,62145526_2,2024-06-06 15:01:00,20240605,10642,NaN,1.717614e+09,y1655,42.28885,-71.12555,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,29.31,NaN,NaN,NaN
2,62145526_2,2024-06-06 15:01:00,20240605,10642,NaN,1.717614e+09,y1655,42.28885,-71.12555,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,29.31,NaN,NaN,NaN
3,62145526_2,2024-06-06 15:01:00,20240605,10642,NaN,1.717614e+09,y1655,42.28885,-71.12555,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,29.31,NaN,NaN,NaN
4,62145526_2,2024-06-06 15:01:00,20240605,10642,NaN,1.717614e+09,y1655,42.28885,-71.12555,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,29.31,NaN,NaN,NaN


In [21]:
df[df['stop_id']==df['id']]

,trip_id,start_time,start_date,stop_id,arrival,departure,vehicle_id,latitude,longitude,bearing,...,precipitationProbability,iceAccumulation,rainAccumulation,sleetAccumulation,snowAccumulation,snowDepth,temperature,id,location_type,vehicle_type
